# grupo6-audio

Essa branch é derivada da "grupo6". É aqui onde fica a subclasse para transcrição de áudios. Coloquei para ele reconhecer tanto áudios enviados pelo Telegram quanto arquivos externos.

In [ ]:
from grp6_bot import BotTelegram # Essa linha importa a classe mãe do arquivo grp6_bot.py
# Se você quiser trabalhar com esse arquivo direto no Colab, você precisa jogar esse arquivo nas Files
from vosk import Model, KaldiRecognizer
import wave
import tempfile
import json
import subprocess

class BotAudio(BotTelegram):

    async def tratar_audio(self, update, context): # Aqui entra o polimorfismo na classe filha

        # Identifica se é áudio do Telegram ou externo:
        if update.message.voice: # Telegram
          file = await update.message.voice.get_file()
          suffix = ".ogg"

        elif update.message.audio: # Externo
          mime = update.message.audio.mime_type # Pega o tipo do arquivo
          if not mime or not mime.startswith("audio/"): # Se o arquivo não é áudio, rejeita
            await update.message.reply_text("Hmm... Parece que o arquivo que você enviou não é um arquivo de áudio válido. Tente novamente!")
            return

          file = await update.message.audio.get_file()
          suffix = os.path.splitext(update.message.audio.file_name)[1] # Pega formato do arquivo

        else:
          await update.message.reply_text("Muito bem! Envie uma mensagem de voz ou um arquivo de áudio para que eu os transcreva!")
          return

        # Salva temporariamente para o processamento:
        temp_audio = tempfile.NamedTemporaryFile(delete=False, suffix=suffix).name
        await file.download_to_drive(temp_audio)

        # Converte para WAV padrão:
        wav_path = temp_audio.replace(suffix, ".wav")
        subprocess.run([
            "ffmpeg", # O FFMPEG é um "programa" que realiza conversões de arquivos
            "-i", temp_audio, # Define arquivo de entrada
            "-ar", "16000", # Taxa de amostragem do áudio de 16kHZ
            "-ac", "1", # Define áudio como mono
            wav_path,
            "-y" # Muda o formato
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Abre arquivo WAV convertido:
        wf = wave.open(wav_path, "rb")

        # Estabelece o modelo do Vosk PT-BR (aqui ele chama o mais leve, podemos trocar depois):
        model = Model("vosk-model-small-pt-0.3")
        rec = KaldiRecognizer(model, wf.getframerate())

        # Processa o áudio:
        while True:
            data = wf.readframes(4000) # Processa 4k frames por vez
            if len(data) == 0: # Espera áudio terminar
                break
            rec.AcceptWaveform(data) # Envia para processamento

        texto = json.loads(rec.FinalResult()).get("text", "") # Extrai o texto do Vosk

        # Verifica se o áudio é vazio; se não, transcreve:
        if not texto.strip():
          await update.message.reply_text("Poxa, seu áudio está vazio :(\nQue tal me enviar algo com conteúdo para que eu possa transcrevê-lo?")
        else:
          await update.message.reply_text(f"Muito bem! Aqui está o seu áudio transcrito:\n{texto}")

